In [1]:
import sys
sys.path.append('/Users/camillecu/Downloads/KUL/master_thesis/master_thesis_github')

In [2]:

# import pysparnn.cluster_index as ci
# import scipy.sparse
# import os
import math
import numpy as np
import pandas as pd
import random
import scipy.sparse as sp
from utils import train_test_split, df_to_matrix ,matrix_to_df_2, threshold_interactions_df, matrix_to_df,set_intersection,get_0_and_p_index,set_diff, matrix_to_full_df, threshold_interactions_df_plus, train_test_split_csr

!pip install surprise
from surprise import Reader, accuracy
from surprise import SVD
from surprise import Dataset
from collections import Counter
from collections import defaultdict

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

from pct.tree.heuristic.Heuristic import Heuristic5
from pct.tree.heuristic.NumericHeuristic_pair import NumericHeuristic5
from pct.tree.splitter.splitter_yahoo import Splitter
import pct.tree.splitter.splitter_yahoo as splitter_yahoo

from pct.tree.Yahootree import Tree


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [12]:
# import df_pairwise from csv
df_pairwise = pd.read_csv('/Users/camillecu/Downloads/KUL/master_thesis/master_thesis_github/yahoo_data/df_pairwise.csv')
# df_pairwise_subset = df_pairwise.sample(n=2000, random_state=42)

# Pivot to create user-item rating matrix
# rating_matrix = df_pairwise_subset.pivot(index='user_id',columns='item_id',values='rating').fillna(0)
rating_matrix = df_pairwise.pivot(index='user_id',columns='item_id',values='rating').fillna(0)
# shape of the rating matrix
rating_matrix.shape

# Create item_type mapping (for splitter logic)
item_type_map = df_pairwise.set_index('item_id')['item_type'].to_dict()
# Assign it to the splitter_yahoo module's variable
splitter_yahoo.item_type_map = item_type_map


In [9]:
df_pairwise.shape

(1524894, 4)

In [15]:
# find all users that rated item 160961
item_id = 160961
users_rated_item = df_pairwise[df_pairwise['item_id'] == item_id]['user_id'].unique()
print(f"Users who rated item {item_id}: {users_rated_item}")
# Create a DataFrame for these users and their ratings
# df_users_rated_item = df_pairwise_subset[df_pairwise_subset['user_id'].isin(users_rated_item)]
# df_users_rated_item

Users who rated item 160961: [     9    137    217 ... 248130 248269 248947]


In [16]:
item_id = 157933
users_rated_item = df_pairwise[df_pairwise['item_id'] == item_id]['user_id'].unique()
print(f"Users who rated item {item_id}: {users_rated_item}")

Users who rated item 157933: [   562   1739   1821   1936   2647   2791   3199   3485   4656   5432
   5891   6033   6055   6616   6768   7141   7532   7969   9675   9942
   9973  10040  10419  10522  10863  11199  11626  11716  12079  12216
  12603  12865  13787  14375  14817  15064  17480  18017  19007  19047
  19544  19777  19869  20049  20206  20380  20848  20873  21036  21057
  21718  21810  21874  22082  22621  23115  23376  23760  23832  24737
  24870  24992  25255  25493  25520  26352  26656  26977  27197  28227
  28639  29992  30169  30850  31081  31421  31484  31540  31683  31768
  33070  33098  33222  34335  35236  35280  35484  35963  36873  37216
  37673  38178  38298  38804  38852  39046  39260  39366  39410  39476
  39518  39665  40383  40876  41143  41594  42339  42459  42669  42782
  42888  43045  43966  44363  44763  45146  45978  46839  47211  47370
  47408  47554  47610  48150  48750  49148  49420  50260  50878  51813
  51906  52121  52513  52600  52898  53382  5397

In [17]:
tree_strategy1 = Tree(min_instances=7, max_depth=3)
tree_strategy1.fit(rating_matrix, df_pairwise['rating'], target_weights=None, strategy=1)

✅Initializing Splitter...
✅ Calling build()...
🔄 Building tree at depth 0 with 1701 users and 1524894 items.

[DEBUG] Ranked items by error: [(132285, '116008.311'), (227957, '116054.692'), (286938, '116151.904'), (237418, '116505.986'), (79867, '116527.627'), (830, '116556.677'), (277464, '116561.038'), (187518, '116584.231'), (103916, '116638.295'), (116038, '116660.260'), (252460, '116668.398'), (61316, '116669.529'), (85921, '116681.638'), (191898, '116724.341'), (210805, '116764.493'), (187696, '116795.749'), (252074, '116805.980'), (247758, '116830.892'), (196502, '116849.066'), (110385, '116870.167'), (189015, '116933.930'), (180964, '116944.818'), (61547, '116960.044'), (172223, '116972.908'), (254836, '117027.291'), (293119, '117029.717'), (278591, '117051.777'), (100355, '117052.385'), (293278, '117105.013'), (194206, '117118.874'), (214565, '117127.494'), (115732, '117187.914'), (183223, '117196.027'), (210727, '117201.978'), (144732, '117206.183'), (212630, '117224.929'), (

In [18]:
tree_strategy2 = Tree(min_instances=15, max_depth=3)
tree_strategy2.fit(rating_matrix, df_pairwise['rating'], target_weights=None, strategy=2)

✅Initializing Splitter...
✅ Calling build()...
🔄 Building tree at depth 0 with 1701 users and 1524894 items.

[DEBUG] Ranked items by error: [(132285, '116008.311'), (227957, '116054.692'), (286938, '116151.904'), (237418, '116505.986'), (79867, '116527.627'), (830, '116556.677'), (277464, '116561.038'), (187518, '116584.231'), (103916, '116638.295'), (116038, '116660.260'), (252460, '116668.398'), (61316, '116669.529'), (85921, '116681.638'), (191898, '116724.341'), (210805, '116764.493'), (187696, '116795.749'), (252074, '116805.980'), (247758, '116830.892'), (196502, '116849.066'), (110385, '116870.167'), (189015, '116933.930'), (180964, '116944.818'), (61547, '116960.044'), (172223, '116972.908'), (254836, '117027.291'), (293119, '117029.717'), (278591, '117051.777'), (100355, '117052.385'), (293278, '117105.013'), (194206, '117118.874'), (214565, '117127.494'), (115732, '117187.914'), (183223, '117196.027'), (210727, '117201.978'), (144732, '117206.183'), (212630, '117224.929'), (

In [19]:
tree_strategy3 = Tree(min_instances=15, max_depth=3)
tree_strategy3.fit(rating_matrix, df_pairwise['rating'], target_weights=None, strategy=3)

✅Initializing Splitter...
✅ Calling build()...
🔄 Building tree at depth 0 with 1701 users and 1524894 items.

[DEBUG] Ranked items by error: [(132285, '116008.311'), (227957, '116054.692'), (286938, '116151.904'), (237418, '116505.986'), (79867, '116527.627'), (830, '116556.677'), (277464, '116561.038'), (187518, '116584.231'), (103916, '116638.295'), (116038, '116660.260'), (252460, '116668.398'), (61316, '116669.529'), (85921, '116681.638'), (191898, '116724.341'), (210805, '116764.493'), (187696, '116795.749'), (252074, '116805.980'), (247758, '116830.892'), (196502, '116849.066'), (110385, '116870.167'), (189015, '116933.930'), (180964, '116944.818'), (61547, '116960.044'), (172223, '116972.908'), (254836, '117027.291'), (293119, '117029.717'), (278591, '117051.777'), (100355, '117052.385'), (293278, '117105.013'), (194206, '117118.874'), (214565, '117127.494'), (115732, '117187.914'), (183223, '117196.027'), (210727, '117201.978'), (144732, '117206.183'), (212630, '117224.929'), (

In [20]:
# build tree structure
tree_strategy1.print_tree_structure()

Leaf Node: No
Depth: 0
Item_ID: (132285, 227957)
user_ids: [9, 137, 217, 510, 562, 600, 624, 917, 1191, 1311, 1483, 1533, 1739, 1821, 1876, 1936, 2033, 2647, 2791, 2857, 3067, 3091, 3199, 3485, 3575, 3582, 3664, 3948, 4656, 4706, 5021, 5036, 5417, 5431, 5432, 5479, 5696, 5782, 5815, 5832, 5891, 6033, 6055, 6069, 6548, 6616, 6701, 6768, 6946, 6953, 7069, 7141, 7194, 7354, 7412, 7461, 7532, 7753, 7969, 8259, 8797, 9466, 9675, 9775, 9942, 9973, 10040, 10097, 10098, 10307, 10341, 10419, 10522, 10547, 10699, 10863, 11177, 11199, 11477, 11562, 11626, 11634, 11716, 11820, 12079, 12130, 12216, 12359, 12603, 12694, 12865, 12936, 13308, 13432, 13787, 13956, 14029, 14375, 14765, 14781, 14817, 15063, 15064, 15480, 16256, 16343, 16846, 16847, 16895, 16975, 16983, 17289, 17305, 17328, 17480, 17865, 17960, 18017, 18059, 18166, 18304, 18908, 19007, 19023, 19047, 19118, 19544, 19677, 19746, 19777, 19869, 19894, 20049, 20206, 20380, 20606, 20829, 20848, 20873, 21036, 21042, 21057, 21267, 21525, 21640, 2

In [21]:
tree_strategy2.print_tree_structure()

Leaf Node: No
Depth: 0
Item_ID: (132285, 85921)
user_ids: [9, 137, 217, 510, 562, 600, 624, 917, 1191, 1311, 1483, 1533, 1739, 1821, 1876, 1936, 2033, 2647, 2791, 2857, 3067, 3091, 3199, 3485, 3575, 3582, 3664, 3948, 4656, 4706, 5021, 5036, 5417, 5431, 5432, 5479, 5696, 5782, 5815, 5832, 5891, 6033, 6055, 6069, 6548, 6616, 6701, 6768, 6946, 6953, 7069, 7141, 7194, 7354, 7412, 7461, 7532, 7753, 7969, 8259, 8797, 9466, 9675, 9775, 9942, 9973, 10040, 10097, 10098, 10307, 10341, 10419, 10522, 10547, 10699, 10863, 11177, 11199, 11477, 11562, 11626, 11634, 11716, 11820, 12079, 12130, 12216, 12359, 12603, 12694, 12865, 12936, 13308, 13432, 13787, 13956, 14029, 14375, 14765, 14781, 14817, 15063, 15064, 15480, 16256, 16343, 16846, 16847, 16895, 16975, 16983, 17289, 17305, 17328, 17480, 17865, 17960, 18017, 18059, 18166, 18304, 18908, 19007, 19023, 19047, 19118, 19544, 19677, 19746, 19777, 19869, 19894, 20049, 20206, 20380, 20606, 20829, 20848, 20873, 21036, 21042, 21057, 21267, 21525, 21640, 21

In [22]:
tree_strategy3.print_tree_structure()

Leaf Node: No
Depth: 0
Item_ID: (132285, 286938)
user_ids: [9, 137, 217, 510, 562, 600, 624, 917, 1191, 1311, 1483, 1533, 1739, 1821, 1876, 1936, 2033, 2647, 2791, 2857, 3067, 3091, 3199, 3485, 3575, 3582, 3664, 3948, 4656, 4706, 5021, 5036, 5417, 5431, 5432, 5479, 5696, 5782, 5815, 5832, 5891, 6033, 6055, 6069, 6548, 6616, 6701, 6768, 6946, 6953, 7069, 7141, 7194, 7354, 7412, 7461, 7532, 7753, 7969, 8259, 8797, 9466, 9675, 9775, 9942, 9973, 10040, 10097, 10098, 10307, 10341, 10419, 10522, 10547, 10699, 10863, 11177, 11199, 11477, 11562, 11626, 11634, 11716, 11820, 12079, 12130, 12216, 12359, 12603, 12694, 12865, 12936, 13308, 13432, 13787, 13956, 14029, 14375, 14765, 14781, 14817, 15063, 15064, 15480, 16256, 16343, 16846, 16847, 16895, 16975, 16983, 17289, 17305, 17328, 17480, 17865, 17960, 18017, 18059, 18166, 18304, 18908, 19007, 19023, 19047, 19118, 19544, 19677, 19746, 19777, 19869, 19894, 20049, 20206, 20380, 20606, 20829, 20848, 20873, 21036, 21042, 21057, 21267, 21525, 21640, 2

In [3]:

my_seed = 7
random.seed(my_seed)
np.random.seed(my_seed)

def load_user_item_matrix(filepath):
    data = []

    with open(filepath, 'r') as f:
        while True:
            user_line = f.readline()
            if not user_line:
                break

            user_line = user_line.strip()
            if '|' not in user_line:
                continue

            user_id, num_ratings = user_line.split('|')
            user_id = int(user_id)
            num_ratings = int(num_ratings)

            for _ in range(num_ratings):
                item_line = f.readline().strip()
                parts = item_line.split('\t')
                if len(parts) < 2:
                    continue
                item_id, score = parts[0], parts[1]
                data.append([user_id, int(item_id), int(score)])

    df = pd.DataFrame(data, columns=['user_id', 'item_id', 'rating'])
    return df


In [4]:
def safe_int(x):
    return int(x) if x != 'None' else 0


def load_track_data(filepath):
    track_data = []
    with open(filepath, 'r') as f:
        for line in f:
            parts = line.strip().split('|')
            track_id = safe_int(parts[0])
            album_id = safe_int(parts[1])
            artist_id = safe_int(parts[2])
            genre_ids = [safe_int(g) for g in parts[3:]] if len(parts) > 3 else []
            track_data.append([track_id, album_id, artist_id, genre_ids])
    return pd.DataFrame(track_data, columns=['track_id', 'album_id', 'artist_id', 'genre_ids'])

def load_album_data(filepath):
    album_data = []
    with open(filepath, 'r') as f:
        for line in f:
            parts = line.strip().split('|')
            album_id = safe_int(parts[0])
            artist_id = safe_int(parts[1])
            genre_ids = [safe_int(g) for g in parts[2:]] if len(parts) > 2 else []
            album_data.append([album_id, artist_id, genre_ids])
    return pd.DataFrame(album_data, columns=['album_id', 'artist_id', 'genre_ids'])


# read artistData2
def load_artist_data(filepath):
    with open(filepath, 'r') as f:
        artist_ids = [int(line.strip()) for line in f]
    return pd.DataFrame(artist_ids, columns=['artist_id'])

# read genreData2
def load_genre_data(filepath):
    with open(filepath, 'r') as f:
        genre_ids = [int(line.strip()) for line in f]
    return pd.DataFrame(genre_ids, columns=['genre_id'])

In [5]:
track_df = load_track_data('/Users/camillecu/Downloads/KUL/master_thesis/master_thesis_github/yahoo_data/ydata-ymusic-kddcup-2011-track2/trackData2.txt')
track_df

,track_id,album_id,artist_id,genre_ids
0,1,106710,281667,"[214765, 162234, 155788]"
1,2,280977,233685,"[131552, 173467, 48505]"
2,3,38422,219136,"[61215, 201738, 88853]"
3,4,119529,166863,"[17453, 35389]"
4,5,16742,294690,"[61215, 34486, 274088]"
...,...,...,...,...
224036,296100,166516,33011,"[274088, 199606, 88853]"
224037,296101,0,0,[]
224038,296102,153644,289056,"[158282, 139095, 242383]"
224039,296105,68336,6613,[82064]


In [6]:
album_df = load_album_data('/Users/camillecu/Downloads/KUL/master_thesis/master_thesis_github/yahoo_data/ydata-ymusic-kddcup-2011-track2/albumData2.txt')
album_df

,album_id,artist_id,genre_ids
0,0,0,[214765]
1,6,228091,"[158282, 81520, 242383]"
2,19,85028,[103715]
3,30,16832,[31567]
4,33,26330,"[149962, 209270]"
...,...,...,...
52824,296104,125866,[158282]
52825,296106,221637,"[116130, 9432]"
52826,296107,0,[61215]
52827,296108,93277,[61215]


In [7]:
artist_df = load_artist_data('/Users/camillecu/Downloads/KUL/master_thesis/master_thesis_github/yahoo_data/ydata-ymusic-kddcup-2011-track2/artistData2.txt')
artist_df

,artist_id
0,16
1,23
2,35
3,40
4,49
...,...
18669,295987
18670,296007
18671,296012
18672,296013


In [8]:
genre_df = load_genre_data('/Users/camillecu/Downloads/KUL/master_thesis/master_thesis_github/yahoo_data/ydata-ymusic-kddcup-2011-track2/genreData2.txt')
genre_df

,genre_id
0,208
1,315
2,642
3,1075
4,1271
...,...
562,292093
563,293670
564,293688
565,294138


In [9]:
train_path = '/Users/camillecu/Downloads/KUL/master_thesis/master_thesis_github/yahoo_data/ydata-ymusic-kddcup-2011-track2/trainIdx2.txt'
my_seed = 7
random.seed(my_seed)
np.random.seed(my_seed)
train_df = load_user_item_matrix(train_path)


In [10]:
train_df

,user_id,item_id,rating
0,0,28341,90
1,0,51210,90
2,0,79500,90
3,0,82317,90
4,0,98399,90
...,...,...,...
61944401,249011,270557,90
61944402,249011,273574,90
61944403,249011,286938,90
61944404,249011,287681,80


In [11]:
duplicate_counts = train_df.groupby(['user_id', 'item_id']).size()
duplicates = duplicate_counts[duplicate_counts > 1]

print(f"🔁 Number of duplicated (user_id, item_id) pairs: {len(duplicates)}")
print(duplicates.head())


🔁 Number of duplicated (user_id, item_id) pairs: 0
Series([], dtype: int64)


In [12]:
print(train_df['item_id'].max())      # highest item_id
print(track_df['track_id'].max())     # highest track_id
print(train_df['item_id'].isin(track_df['track_id']).mean())  # proportion to match

print(train_df['item_id'].isin(album_df['album_id']).mean())

print(train_df['item_id'].isin(artist_df['artist_id']).mean())

print(train_df['item_id'].isin(genre_df['genre_id']).mean())



296110
296110
0.4385845107627636
0.1925648621120041
0.3114063600835885
0.05744426704164376


In [13]:
def build_item_metadata(track_df, album_df, artist_df, genre_df):
    rows = []

    # 1. Tracks
    for _, row in track_df.iterrows():
        rows.append({
            'item_id': row['track_id'],
            'item_type': 'track',
            'artist_id': row['artist_id'],
            'genre_ids': row['genre_ids']
        })

    # 2. Albums
    for _, row in album_df.iterrows():
        rows.append({
            'item_id': row['album_id'],
            'item_type': 'album',
            'artist_id': row['artist_id'],
            'genre_ids': row['genre_ids']
        })

    # 3. Artists
    for artist_id in artist_df['artist_id']:
        rows.append({
            'item_id': artist_id,
            'item_type': 'artist',
            'artist_id': artist_id,
            'genre_ids': 0
        })

    # 4. Genres
    for genre_id in genre_df['genre_id']:
        rows.append({
            'item_id': genre_id,
            'item_type': 'genre',
            'artist_id': 0,
            'genre_ids': [genre_id]
        })

    return pd.DataFrame(rows)

In [14]:
item_meta_df = build_item_metadata(track_df, album_df, artist_df, genre_df)

train_with_meta = train_df.merge(item_meta_df, on='item_id', how='inner')

# When loading the data, sort by a stable column (e.g., user_id)
train_with_meta = train_with_meta.sort_values('user_id').reset_index(drop=True)

train_with_meta

,user_id,item_id,rating,item_type,artist_id,genre_ids
0,0,28341,90,artist,28341,0
1,0,232604,30,artist,232604,0
2,0,237425,50,artist,237425,0
3,0,245257,50,artist,245257,0
4,0,23301,50,artist,23301,0
...,...,...,...,...,...,...
61944401,249011,141075,90,artist,141075,0
61944402,249011,150385,80,genre,0,[150385]
61944403,249011,152077,90,artist,152077,0
61944404,249011,158282,80,genre,0,[158282]


In [15]:
df = train_with_meta.copy()

# Filter: keep only artist & genre (drop album, track)
df = df[~df['item_type'].isin(['album', 'track'])].copy()

print(f"🎯 Sample shape: {df.shape}")
print(f"👤 Users: {df['user_id'].nunique()}, 🎵 Items: {df['item_id'].nunique()}")

🎯 Sample shape: (22848233, 6)
👤 Users: 248344, 🎵 Items: 19241


In [16]:
filtered_df = threshold_interactions_df(df,'user_id','item_id',100,500)

Starting interactions info
Number of rows: 248344
Number of cols: 19241
Sparsity: 0.478%
Ending interactions info
Number of rows: 4644
Number of columns: 8938
Sparsity: 10.674%


In [17]:
# filtered_df = threshold_interactions_df_plus(df, min_items_per_user=500, min_artists_per_user=20, min_genres_per_user=10,min_users_per_item=150)
my_seed = 7
random.seed(my_seed)
np.random.seed(my_seed)
sampled_users = np.random.choice(filtered_df['user_id'].unique(), size=2000, replace=False)
sampled_df = filtered_df[filtered_df['user_id'].isin(sampled_users)]
print(f"🎯 Sample shape: {sampled_df.shape}")
print(f"👤 Users: {sampled_df['user_id'].nunique()}, 🎵 Items: {sampled_df['item_id'].nunique()}")
filtered_df = threshold_interactions_df(sampled_df,'user_id','item_id',100,500)

🎯 Sample shape: (1941009, 6)
👤 Users: 2000, 🎵 Items: 8938
Starting interactions info
Number of rows: 2000
Number of cols: 8938
Sparsity: 10.858%
Ending interactions info
Number of rows: 1701
Number of columns: 5011
Sparsity: 17.890%


Sample shape: (1941009, 6)
👤 Users: 2000, 🎵 Items: 8938
Starting interactions info
Number of rows: 2000
Number of cols: 8938
Sparsity: 10.858%
Ending interactions info
Number of rows: 1701
Number of columns: 5011
Sparsity: 17.890%

In [18]:
filtered_df

,user_id,item_id,rating,item_type,artist_id,genre_ids
1039,9,182744,30,artist,182744,0
1040,9,197011,50,artist,197011,0
1041,9,160820,0,artist,160820,0
1043,9,59402,30,artist,59402,0
1044,9,251901,0,artist,251901,0
...,...,...,...,...,...,...
61933948,248947,210727,50,artist,210727,0
61933956,248947,275456,0,artist,275456,0
61933959,248947,203741,0,artist,203741,0
61933965,248947,202602,0,artist,202602,0


# Test Yahoo tree build

In [19]:
# convert ratings to 1 when rating > 50; if rating <= 50 but greater than 0, set to 0.1, otherwise set to 0
df_pairwise = filtered_df.copy()
df_pairwise['rating'] = df_pairwise['rating'].apply(lambda x: 1 if x > 50 else (0.1 if x > 0 else 0))


In [23]:
# drop genre_ids and artist_id columns
df_pairwise = df_pairwise.drop(columns=['genre_ids', 'artist_id'])

In [25]:
# export df_pairwise to csv
df_pairwise.to_csv('/Users/camillecu/Downloads/KUL/master_thesis/master_thesis_github/yahoo_data/df_pairwise.csv', index=False)
# convert df_pairwise to matrix

In [ ]:
# import df_pairwise from csv
df_pairwise = pd.read_csv('/Users/camillecu/Downloads/KUL/master_thesis/master_thesis_github/yahoo_data/df_pairwise.csv')
df_pairwise_subset = df_pairwise.sample(n=2000, random_state=42)


In [4]:
df_pairwise.head()

,user_id,item_id,rating,item_type
0,9,182744,0.1,artist
1,9,197011,0.1,artist
2,9,160820,0.0,artist
3,9,59402,0.1,artist
4,9,251901,0.0,artist


In [7]:
# type of value of rating column
print(df_pairwise['rating'].dtype)

float64


In [5]:
# Pivot to create user-item rating matrix
rating_matrix = df_pairwise_subset.pivot(
    index='user_id',
    columns='item_id',
    values='rating'
).fillna(0)

# Create item_type mapping (for splitter logic)
item_type_map = df_pairwise_subset.set_index('item_id')['item_type'].to_dict()


In [5]:
tree = Tree(min_instances=15)
tree.fit(df_pairwise_subset, df_pairwise_subset['rating'], target_weights=None, strategy=1)

✅ Numerical attributes: Index(['user_id', 'item_id', 'rating'], dtype='object')
✅ Categorical attributes: Index(['item_type'], dtype='object')
Initializing Splitter...
✅ Calling build()...


/Users/camillecu/Downloads/KUL/master_thesis/master_thesis_github/thesis_github/lib/python3.9/site-packages/numpy/core/fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


TypeError: unsupported operand type(s) for ** or pow(): 'str' and 'int'

In [ ]:
all_user_ids = sorted(filtered_df['user_id'].unique())

def split_users_by_ratio(all_user_ids, ratio):
    n = len(all_user_ids)
    split_point = int(n * ratio)
    warm_users = all_user_ids[:split_point]
    cold_users = all_user_ids[split_point:]
    return warm_users, cold_users

# Example ratios from 10% to 50%
# ratios = [0.1, 0.2, 0.3, 0.4, 0.5]
# splits = {r: split_users_by_ratio(all_user_ids, r) for r in ratios}


## 10% warm

In [ ]:
warm_users_idx, cold_users_idx = split_users_by_ratio(all_user_ids, 0.1)

df_warm = filtered_df[filtered_df['user_id'].isin(warm_users_idx)].copy()
df_cold = filtered_df[filtered_df['user_id'].isin(cold_users_idx)].copy()

matrix_warm, rid_to_idx_warm, idx_to_rid_warm, cid_to_idx, idx_to_cid = df_to_matrix(
    df_warm, "user_id", "item_id", "rating")


matrix_cold, rid_to_idx_cold, idx_to_rid_cold, _, _ = df_to_matrix( 
    df_cold, "user_id", "item_id", "rating")



In [ ]:
def split_and_combine(strategy="artist-only"):
    """Handles both approaches with proper matrix alignment"""
    # Get full cold matrix and mappings
    matrix_cold, rid_to_idx_cold, _, cid_to_idx, _ = df_to_matrix(
        df_cold, "user_id", "item_id", "rating"
    )
    matrix_cold = matrix_cold.tocsr()

    # Create boolean masks
    artist_mask = np.isin(
        np.arange(matrix_cold.shape[1]), 
        [cid_to_idx[iid] for iid in df_cold[df_cold['item_type'] == 'artist']['item_id']]
    )
    genre_mask = ~artist_mask

    # Create aligned matrices
    matrix_cold_artist = matrix_cold.multiply(artist_mask)
    matrix_cold_genre = matrix_cold.multiply(genre_mask)
    matrix_cold_artist = matrix_cold.multiply(artist_mask).tocsr()
    matrix_cold_genre = matrix_cold.multiply(genre_mask).tocsr()

    al_artist, test_cold, _ = train_test_split(
        matrix_cold_artist, 
        split_count=30,
        fraction=None
    )
    
    if strategy == "artist-only":
        X_cold, K_cold, _ = train_test_split_csr(al_artist, 1)  
        return K_cold, X_cold, test_cold
    
    elif strategy == "hybrid":
        X_cold, K_cold, _ = train_test_split_csr(al_artist, 1)
        X_cold = X_cold + matrix_cold_genre
        return K_cold, X_cold, test_cold


In [ ]:
my_seed = 7
random.seed(my_seed)
np.random.seed(my_seed)

train_cold_K_artist, X_cold_artist, test_cold_artist  = split_and_combine("artist-only")
train_cold_K_hybrid, X_cold_hybrid, test_cold_hybrid = split_and_combine("hybrid")


In [ ]:
# shape of test_cold

print(f"Cold users in X: {len(np.unique(X_cold_artist.nonzero()[0]))}")
print(f"Cold items in X: {len(np.unique(X_cold_artist.nonzero()[1]))}")
print(f"test users in test: {len(np.unique(test_cold_artist.nonzero()[0]))}")
print(f"test items in test: {len(np.unique(test_cold_artist.nonzero()[1]))}")
print(f"train users in train: {len(np.unique(train_cold_K_artist.nonzero()[0]))}")
print(f"train items in train: {len(np.unique(train_cold_K_artist.nonzero()[1]))}")

print("-------------------")

# shape of test_cold
print(f"Shape of test_cold: {test_cold_artist.shape}")
print(f"Shape of train_cold: {train_cold_K_artist.shape}")
print(f"Shape of X_cold: {X_cold_artist.shape}")

Cold users in X: 1531
Cold items in X: 4793
test users in test: 1473
test items in test: 3890
train users in train: 665
train items in train: 576
-------------------
Shape of test_cold: (1531, 5011)
Shape of train_cold: (1531, 5011)
Shape of X_cold: (1531, 5011)
